In [1]:
%设置scattering wavelet transform 的参数
filt_opt.J = 3;
filt_opt.L1 = 2;
filt_opt.L2 = 2;
scat_opt.M = 2;
%读取通过python预处理后的CT序列
load('CT_32_seg')
%读取训练集的标签
load('label_train')

In [40]:
%对训练集进行scattering wavelet transform 并提取子带的特征 分解得到37个子带 每个子带4个特征 共148个特征
tic;
for i=1:465
    data=reshape(double(train(i,:,:,:))/255,[32,32,32]);
    img=data;
    %设置scaterring transform的参数   
    Wop = my_wavelet_factory_3d(size(img),filt_opt, scat_opt);
    Sx = scat(img, Wop);
    %进行变换 得到37个子带
    S = format_scat(Sx);
    for j=1:37

        SS=S(j,:,:,:);
        EE=SS(:).^2;
        %feature1 均值
        F1(j)=mean(SS(:));
        %feature2 能量
        F2(j)=norm(SS(:),2)^2;
        %feature3 标准差
        F3(j)=std(SS(:));
        %feature4 熵
        p=EE./sum(EE);
        F4(j)=-sum(p.*log2(p));
    end
     X_train(i,:)=[F1 F2 F3 F4];
    end

toc;
disp("训练集图像特征提取完毕")
disp("显示第一个结节的特征")
disp(X_train(1,:))

时间已过 129.351718 秒。
训练集图像特征提取完毕
显示第一个结节的特征
  1 至 16 列

    0.0563    0.0016    0.0015    0.0016    0.0046    0.0046    0.0040    0.0168    0.0174    0.0117    0.0004    0.0003    0.0002    0.0009    0.0007    0.0006

  17 至 32 列

    0.0003    0.0004    0.0003    0.0007    0.0009    0.0006    0.0003    0.0003    0.0003    0.0006    0.0007    0.0009    0.0026    0.0020    0.0014    0.0019

  33 至 48 列

    0.0025    0.0015    0.0015    0.0016    0.0018    5.4036    0.0021    0.0019    0.0026    0.0167    0.0165    0.0146    0.2143    0.2287    0.1111    0.0001

  49 至 64 列

    0.0001    0.0000    0.0006    0.0004    0.0002    0.0001    0.0001    0.0001    0.0003    0.0006    0.0003    0.0000    0.0001    0.0001    0.0003    0.0003

  65 至 80 列

    0.0008    0.0050    0.0024    0.0015    0.0022    0.0049    0.0017    0.0014    0.0018    0.0034    0.0860    0.0012    0.0012    0.0015    0.0034    0.0034

  81 至 96 列

    0.0035    0.0117    0.0120    0.0090    0.0003    0.0002    0.0002 

In [64]:
    %不知道为啥 第253个数据有点异常  把它替换成第252个
    X_train(253,:)=X_train(252,:);

In [77]:
%对验证集进行scattering wavelet transform 并提取子带的特征  分解得到37个子带 每个子带4个特征 共148个特征
tic
for i=1:117
    data=reshape(double(test(i,:,:,:))/255,[32,32,32]);
    img=data;    
    Wop = my_wavelet_factory_3d(size(img),filt_opt, scat_opt);
    Sx = scat(img, Wop);
    S = format_scat(Sx);
    for j=1:37
        SS=S(j,:,:,:);
        EE=SS(:).^2;
        F1(j)=mean(SS(:));
        F2(j)=norm(SS(:),2)^2;
        F3(j)=std(SS(:));
        p=EE./sum(EE);
        F4(j)=-sum(p.*log2(p));
    end
        X_test(i,:)=[F1 F2 F3 F4];
  end
  toc
  disp("测试集特征提取完毕")


时间已过 41.705558 秒。
测试集特征提取完毕



In [97]:
%对训练集和验证集的148个特征进行归一化处理
for i=1:148
    X_train_norm(:,i)=(X_train(:,i)-mean(X_train(:,i)'))/std(X_train(:,i)');

    X_test_norm(:,i)=(X_test(:,i)-mean(X_train(:,i)'))/std(X_train(:,i)');
end




In [98]:
%第一个模型
m=1
%训练集标签
Y_train=label;
Y_train(253)=Y_train(252);

load('model_1.mat')
%将训练集随机打乱
%c1=randperm(numel(1:len));
%a1=c1(1:floor(len*14/15));
%b1=c1(floor(len*11/15)+1:len);
%从训练集中随机划分一部分作为SVM的训练集
Y_train1=Y_train(a1);
X_train1 = double(X_train_norm(a1,:));
%从训练集中随机划分一部分作为SVM的测试集
Y_test1=Y_train(b1);
X_test1 = double(X_train_norm(b1,:));

%待验证的验证集
X_test2=double(X_test_norm);

%模型的训练  这里使用已经训练好的模型

%model_SVM1 = svmtrain(Y_train1,train_scat,'-c 2172 -g 0.001    -b 1');
disp('训练集准确率')
[predict_label, accuracy, prob] = svmpredict( Y_train1,X_train1 , model_SVM);
disp('测试集准确率')
[predict_label, accuracy, prob] = svmpredict(Y_test1,X_test1, model_SVM,'-b 1');

disp('验证集')
[predict_label, accuracy, prob2] = svmpredict(Y_train1(1:117) ,X_test2, model_SVM,'-b 1');




m =

     1

训练集准确率
Model supports probability estimates, but disabled in predicton.
Accuracy = 79.4931% (345/434) (classification)
测试集准确率
Accuracy = 74.1935% (23/31) (classification)
验证集
Accuracy = 38.4615% (45/117) (classification)



In [99]:
disp('第一个模型的测试集AUC')
 x=AUC(Y_test1,prob(:,1)')

第一个模型的测试集AUC

x =

    0.7815




In [79]:
disp('第一个模型输出的验证集概率prob')
disp(prob2(:,1))

第一个模型输出的验证集概率prob
    0.4870
    0.2603
    0.5263
    0.6620
    0.4510
    0.4947
    0.6112
    0.6628
    0.5594
    0.5537
    0.4885
    0.5519
    0.4829
    0.5005
    0.6471
    0.4732
    0.4892
    0.4610
    0.5033
    0.4358
    0.4792
    0.5566
    0.5441
    0.4642
    0.5631
    0.5769
    0.4682
    0.1918
    0.4710
    0.6145
    0.5613
    0.4755
    0.5036
    0.4891
    0.6280
    0.6223
    0.4906
    0.5010
    0.5845
    0.5014
    0.5536
    0.4913
    0.5619
    0.6174
    0.4906
    0.4737
    0.6489
    0.6945
    0.4626
    0.4641
    0.4821
    0.5914
    0.4537
    0.4770
    0.6933
    0.5231
    0.3987
    0.6255
    0.5485
    0.5979
    0.5570
    0.4607
    0.4840
    0.5582
    0.5171
    0.4886
    0.6275
    0.5805
    0.4815
    0.3718
    0.4599
    0.5541
    0.5341
    0.6318
    0.6241
    0.5567
    0.6400
    0.4387
    0.5823
    0.5275
    0.4652
    0.3679
    0.5088
    0.4745
    0.5871
    0.5046
    0.4813
    0.4617
    0.5480
   

In [81]:
probb(:,m)=prob2(:,1);

In [100]:
%第二个模型


m=2
%训练集标签
Y_train=label;
Y_train(253)=Y_train(252);

load('model_2.mat')
%将训练集随机打乱
%c1=randperm(numel(1:len));
%a1=c1(1:floor(len*14/15));
%b1=c1(floor(len*11/15)+1:len);
%从训练集中随机划分一部分作为SVM的训练集
Y_train1=Y_train(a1);
X_train1 = double(X_train_norm(a1,:));
%从训练集中随机划分一部分作为SVM的测试集
Y_test1=Y_train(b1);
X_test1 = double(X_train_norm(b1,:));

%待验证的验证集
X_test2=double(X_test_norm);

%模型的训练  这里使用已经训练好的模型

%model_SVM1 = svmtrain(Y_train1,train_scat,'-c 2172 -g 0.001    -b 1');
disp('训练集准确率')
[predict_label, accuracy, prob] = svmpredict( Y_train1,X_train1 , model_SVM2);
disp('测试集准确率')
[predict_label, accuracy, prob] = svmpredict(Y_test1,X_test1, model_SVM2,'-b 1');

disp('验证集')
[predict_label, accuracy, prob2] = svmpredict(Y_train1(1:117) ,X_test2, model_SVM2,'-b 1');




m =

     2

训练集准确率
Model supports probability estimates, but disabled in predicton.
Accuracy = 82.4885% (358/434) (classification)
测试集准确率
Accuracy = 67.7419% (21/31) (classification)
验证集
Accuracy = 42.735% (50/117) (classification)



In [101]:
disp('第一个模型的测试集AUC')
 x=AUC(Y_test1,prob(:,1)')

第一个模型的测试集AUC

x =

    0.7899




In [86]:
disp('第二个模型输出的验证集概率prob')
disp(prob2(:,1))

第二个模型输出的验证集概率prob
    0.4441
    0.2477
    0.5286
    0.7585
    0.4286
    0.5639
    0.6114
    0.6528
    0.5606
    0.5857
    0.4475
    0.5520
    0.4533
    0.4873
    0.6542
    0.4341
    0.4591
    0.4439
    0.5077
    0.4500
    0.4594
    0.5380
    0.5634
    0.4993
    0.5739
    0.6103
    0.4449
    0.3571
    0.4697
    0.5062
    0.6189
    0.4457
    0.5184
    0.4664
    0.5930
    0.7383
    0.5106
    0.4953
    0.5975
    0.5514
    0.5991
    0.4122
    0.5809
    0.6268
    0.4621
    0.4452
    0.6373
    0.7189
    0.5542
    0.4419
    0.4737
    0.4829
    0.4529
    0.4599
    0.6455
    0.5258
    0.4924
    0.7085
    0.6282
    0.7078
    0.5436
    0.4707
    0.4652
    0.5707
    0.5535
    0.4624
    0.5526
    0.5569
    0.4409
    0.3002
    0.4819
    0.5861
    0.5357
    0.6252
    0.5136
    0.5776
    0.5813
    0.4344
    0.6433
    0.5302
    0.4425
    0.7453
    0.5028
    0.4540
    0.5923
    0.5054
    0.4530
    0.3980
    0.4517
   

In [95]:
%模型集成得到的最终结果
probb(:,m)=prob2(:,1);
prob=mean(probb')';

In [96]:
%输出csv文件
columns = {'Predicted'};
data = table(prob, 'VariableNames', columns)
writetable(data, 'Submission.csv')


data =

  117x1 table

    Predicted
    _________

     0.46555 
     0.25399 
     0.52746 
     0.71027 
     0.43979 
     0.52927 
      0.6113 
     0.65777 
        0.56 
     0.56973 
       0.468 
      0.5519 
     0.46811 
      0.4939 
     0.65067 
     0.45369 
     0.47418 
     0.45244 
     0.50549 
     0.44293 
     0.46933 
      0.5473 
     0.55379 
     0.48174 
     0.56849 
      0.5936 
     0.45655 
     0.27443 
     0.47031 
     0.56037 
     0.59008 
     0.46057 
       0.511 
     0.47779 
      0.6105 
     0.68029 
     0.50059 
     0.49817 
     0.59096 
      0.5264 
     0.57635 
     0.45171 
     0.57139 
     0.62208 
     0.47633 
     0.45946 
     0.64308 
     0.70668 
      0.5084 
       0.453 
      0.4779 
     0.53717 
     0.45326 
     0.46849 
     0.66937 
     0.52446 
     0.44552 
       0.667 
     0.58834 
     0.65287 
     0.55029 
     0.46573 
     0.47462 
     0.56444 
      0.5353 
     0.47547 
     0.59006 
     0.56